In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import re
    

In [2]:
# Load sample dataset
population_filename = "../data/population-since-10000bc.csv"
population_data = pd.read_csv(population_filename)
popaulation_column = 'Population (historical)' # column with numerical data

population_data.head()

,Entity,Code,Year,Population (historical)
0,Afghanistan,AFG,-10000,14737
1,Afghanistan,AFG,-9000,20405
2,Afghanistan,AFG,-8000,28253
3,Afghanistan,AFG,-7000,39120
4,Afghanistan,AFG,-6000,54166


In [3]:
MIN_ORDERS_OF_MAGNITUDE = 2

def check_magnitude_range(series):
    max_value = series.abs().max()
    min_value = series.abs().min()
    
    magnitude_range = np.log10(max_value) - np.log10(min_value)
    
    return magnitude_range

print(check_magnitude_range(population_data[popaulation_column]))

9.908041643529613


In [18]:
benford_probs = {
    d: np.log10(1 + 1/d) for d in range(1, 10)
}

benford_first_digits = list(benford_probs.values())
benford_last_digits = [0.1] * 10

def test_goodness_of_fit_chisquare(observed_counts, expected, label='first'):
    print(f"\nTesting goodness of fit for {label} digits:")

    # Example observed data: digit counts from your dataset
    observed_counts = np.array(observed_counts)  # replace with your counts
    n = observed_counts.sum()

    # Expected counts under Benford's law
    expected_counts = np.array([p * n for p in expected])
    print(f"observed counts for {label} digits:", observed_counts)
    print(f"expected counts for {label} digits:", expected_counts)

    # Chi-square goodness of fit test
    chi2_stat, p_value = chisquare(f_obs=observed_counts, f_exp=expected_counts)

    print(f"\nChi-square goodness of fit test for {label} digits:")
    print("Chi-square statistic:", chi2_stat)
    print("p-value:", p_value)

    if p_value > 0.05:
        print(f"Fail to reject null: data follows Benford's law (at 5% significance) for {label} digits.")
    else:
        print(f"Reject null: data does not follow Benford's law for {label} digits.")
        
    return chi2_stat, p_value

def test_goodness_of_fit_mad(observed_counts, expected, label='first'):
    observed_counts = np.array(observed_counts)

    # Convert observed counts to proportions
    observed_probs = observed_counts / observed_counts.sum()

    # Mean Absolute Deviation
    mad = np.mean(np.abs(observed_probs - expected))
    print(f"{label.capitalize()} digits - MAD:", mad)

    # Classify conformity (per Nigrini's thresholds)
    if mad < 0.006:
        print(f"{label.capitalize()} digits show Close conformity with Benford's Law")
    elif mad < 0.012:
        print(f"{label.capitalize()} digits show Acceptable conformity with Benford's Law")
    else:
        print(f"{label.capitalize()} digits show Non-conformity with Benford's Law")
        
    return mad


In [17]:
def get_dataset_stats(dataset):
    
    df = pd.read_csv(f'../data/{dataset}')
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f'Found {len(numerical_cols)} numerical columns in {dataset}: {numerical_cols} \n\n')
    
    print('COLUMN-WISE ANALYSIS:')
    for col in numerical_cols:
        print('----------------------------------\n')
        
        
        col_data = df[col].abs()
        col_data = col_data.replace(0, np.nan).dropna()
        
        magnitude_range = np.round(check_magnitude_range(col_data), 2)
        
        print(f'Column "{col}" has a magnitude range of {magnitude_range}.')

        if magnitude_range == MIN_ORDERS_OF_MAGNITUDE:
            print(f'Since magnitude range({magnitude_range} = 2), the data should start to show Benford-like properties.')
            
        elif (magnitude_range > MIN_ORDERS_OF_MAGNITUDE):
            print(f'Since magnitude range({magnitude_range}) > 2, it is a good candidate for Benford analysis.')

        else:
            print(f'Since magnitude range({magnitude_range}) < 2, "{col}" is unlikely to follow Benford\'s Law.')


        # Extract first and last digits
        first_digits = col_data.apply(extract_first_digit).value_counts().sort_index()
        first_digits_ratio = first_digits / first_digits.sum()

        last_digits = col_data.apply(extract_last_digit).value_counts().sort_index()
        last_digits_ratio = last_digits / last_digits.sum()
        
        benford_first_digit_series = pd.Series(benford_first_digits)
        benford_first_digit_series.index = benford_first_digit_series.index
        
        
        results = pd.DataFrame({
            'First Digit Ratio': first_digits_ratio,
            'Expected Benford First Digit Ratio': benford_first_digit_series,
            'Last Digit Ratio': last_digits_ratio,
        })

        display(f'Distribution of first and last digits for column "{col}":')
        display(results.fillna(0).round(4))
        
        # chisquare_first, pvalue_first = test_goodness_of_fit_chisquare(first_digits.values.tolist(), benford_first_digits, label='first')
        # chisquare_last, pvalue_last = test_goodness_of_fit_chisquare(last_digits.values.tolist(), benford_last_digits, label='last')
        mad_first = test_goodness_of_fit_mad(first_digits.values.tolist(), benford_first_digits, label='first')
        mad_last = test_goodness_of_fit_mad(last_digits.values.tolist(), benford_last_digits, label='last')


def extract_first_digit(x):
    s = str(abs(x))  # remove sign
    s = re.sub(r"[^0-9]", "", s)  # keep only digits
    for ch in s:
        if ch != "0":
            return int(ch)
    return None


def extract_last_digit(x):
    """Extract last digit from a number, ignoring trailing .0 if present."""
    try:
        s = str(x).strip()
        
        # If the number ends with ".0", drop it
        if s.endswith(".0"):
            s = s[:-2]
        
        s = s.replace(".", "").replace("-", "")  # remove dot and minus
        return int(s[-1]) if s else None
    
    except Exception:
        return None
    
get_dataset_stats(population_filename)

Found 2 numerical columns in ../data/population-since-10000bc.csv: ['Year', 'Population (historical)'] 


COLUMN-WISE ANALYSIS:
----------------------------------

Column "Year" has a magnitude range of 2.0.
Since magnitude range(2.0 = 2), the data should start to show Benford-like properties.


'Distribution of first and last digits for column "Year":'

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0000,0.3010,0.2209
1,0.8417,0.1761,0.0899
2,0.1120,0.1249,0.0894
3,0.0066,0.0969,0.0896
4,0.0066,0.0792,0.0850
5,0.0066,0.0669,0.0851
6,0.0066,0.0580,0.0851
7,0.0066,0.0512,0.0850
8,0.0066,0.0458,0.0850
9,0.0066,0.0000,0.0850


MAD: 0.12014927566412029
Non-conformity with Benford's Law
MAD: 0.024189255958972967
Non-conformity with Benford's Law
----------------------------------

Column "Population (historical)" has a magnitude range of 9.91.
Since magnitude range(9.91) > 2, it is a good candidate for Benford analysis.


'Distribution of first and last digits for column "Population (historical)":'

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0000,0.3010,0.1386
1,0.2836,0.1761,0.0949
2,0.1863,0.1249,0.0946
3,0.1331,0.0969,0.0973
4,0.1020,0.0792,0.0975
5,0.0809,0.0669,0.0942
6,0.0654,0.0580,0.0961
7,0.0549,0.0512,0.0939
8,0.0488,0.0458,0.0965
9,0.0451,0.0000,0.0966


MAD: 0.005578495913507595
Close conformity with Benford's Law
MAD: 0.007719476991000174
Acceptable conformity with Benford's Law


In [14]:
get_dataset_stats('population-since-10000bc.csv')

Found 2 numerical columns in population-since-10000bc.csv: ['Year', 'Population (historical)'] 


COLUMN-WISE ANALYSIS:
----------------------------------

Column "Year" has a magnitude range of 2.0.
Since magnitude range(2.0 = 2), the data should start to show Benford-like properties.


'Distribution of first and last digits for column "Year":'

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0000,0.3010,0.2209
1,0.8417,0.1761,0.0899
2,0.1120,0.1249,0.0894
3,0.0066,0.0969,0.0896
4,0.0066,0.0792,0.0850
5,0.0066,0.0669,0.0851
6,0.0066,0.0580,0.0851
7,0.0066,0.0512,0.0850
8,0.0066,0.0458,0.0850
9,0.0066,0.0000,0.0850


MAD: 0.12014927566412029
Non-conformity with Benford's Law
MAD: 0.024189255958972967
Non-conformity with Benford's Law
----------------------------------

Column "Population (historical)" has a magnitude range of 9.91.
Since magnitude range(9.91) > 2, it is a good candidate for Benford analysis.


'Distribution of first and last digits for column "Population (historical)":'

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0000,0.3010,0.1386
1,0.2836,0.1761,0.0949
2,0.1863,0.1249,0.0946
3,0.1331,0.0969,0.0973
4,0.1020,0.0792,0.0975
5,0.0809,0.0669,0.0942
6,0.0654,0.0580,0.0961
7,0.0549,0.0512,0.0939
8,0.0488,0.0458,0.0965
9,0.0451,0.0000,0.0966


MAD: 0.005578495913507595
Close conformity with Benford's Law
MAD: 0.007719476991000174
Acceptable conformity with Benford's Law


In [ ]:
test_goodness_of_fit_chisquare([280, 136, 125, 97, 79, 67, 58, 51, 46], benford_first_digits, label='first')


Testing goodness of fit for first digits:
observed counts for first digits: [280 136 125  97  79  67  58  51  46]
expected counts for first digits: [282.66716593 165.34969225 117.31747368  90.99850221  74.35119004
  62.86303546  54.45443821  48.03221858  42.96628364]

Chi-square goodness of fit test for first digits:
Chi-square statistic: 7.325000172917046
p-value: 0.5020076551970704
Fail to reject null: data follows Benford's law (at 5% significance) for first digits.


(7.325000172917046, 0.5020076551970704)

In [16]:
test_goodness_of_fit_mad([280, 136, 125, 97, 79, 67, 58, 51, 46], benford_first_digits, label='first')

MAD: 0.00757705790599054
Acceptable conformity with Benford's Law
